In [ ]:
category = 'FPRO.txt' # or 'MPRO.txt'
import numpy as np
import os

races = os.listdir() # all files in the current directory
races = [k for k in races if category in k] # all files that are 'MPRO' or 'FPRO' files
main = [] # this is were all the data is stored
timelimit = 0 # (possibility to filter by race duration)
datelimit = 751355 # date limit, includes at least the last calendar year and two IM and two 70.3 world championships (current limit 70.3 Worlds 2019)
datelimit2 = 752109 # date limit, includes at least the current calendar year and one IM and one 70.3 world championships (current limit 70.3 Worlds 2021)
for race in races:
    file = open(race)
    lines = file.readlines()
    if len(lines) < 4: # if no data -> continue to next race
        continue
    entry = [race] # add race name
    if int(lines[0].split('\n')[0]) < datelimit: # if older than datelimit -> continue to next race
        continue
    for line in lines:
        entry.append(line.split('\n')[0])
    main.append(entry) # add entry to "main"

athletes = [] # list of all athletes
for entry in main:
    if len(entry) > 3: # if no data -> continue to next race
        if int(entry[2]) < timelimit: # (possibility to filter by race duration)
            continue
        for j in range(3,len(entry)): # all the athletes in the race
            if entry[j] == 'DNF': # ignore DNF
                continue
            if entry[j] not in athletes:
                athletes.append(entry[j]) # if athlete is not in athlete list, add athlete           
athletes = np.sort(athletes)
athletes = list(athletes) # alphabetical list of athletes
resultlist = []
while len(athletes) > 0:
    active = np.ones(len(athletes)) # check if this athlete is possibly the best
    active2 = np.ones(len(athletes)) # all strong athlete that are considered for calculation
    tiebreaker = np.zeros(len(athletes))
    breakcheck = False
    while True: # continue until the strongest athlete is found
        points = np.zeros(len(athletes)) # current points (only positive)
        points2 = np.zeros(len(athletes)) # long-term points (only positive)
        for entry in main: # go over all races
            racedate = int(entry[1])
            j = 3
            while entry[j] not in athletes: # if the race winner is already ranked (in front)
                j = j + 1 # the next-placed athlete is considered the winner
                if j == len(entry)-1 or entry[j] == 'DNF': # no winner left
                    break
            if j == len(entry)-1 or entry[j] == 'DNF': # no winner left
                continue # continue to next race
            winner = athletes.index(entry[j]) # index of the winner in "athletes"
            if active[winner] == 0: # if this winner is already ruled out
                continue # continue to next race

            for j in range(j+1,len(entry)): # go over everyone that was beaten
                if entry[j] not in athletes: # skip 'DNF' marking or athletes that aren't in "athletes" anymore
                    continue
                loser = athletes.index(entry[j]) # index of the beaten athlete
                if active2[loser] == 0: # skip athletes that aren't considered strong
                    continue
                if racedate >= datelimit2: # short-term
                    points[winner] = points[winner] + 1 # winner gets a point
                    points[loser] = points[loser] - 1
                points2[winner] = points2[winner] + 1 # long term
                points2[loser] = points2[loser] - 1
        for i in np.nonzero(active)[0]: # go over all possible candidates
            tiebreaker[i] = tiebreaker[i]/100 + min(9,points[i]) + min(9,points2[i])/10
            if points[i] < 4:
                active[i] = 0
        for i in np.nonzero(active2)[0]: # go over all strong opponents
            if points[i] < 1 and (points[i] < 0 or points2[i] < 1):
                active2[i] = 0
            active2[i] = active[i]
        pointslimit = 3
        points2limit = 7
        while sum(active) == 0: # in case there are no candidates left who fulfill all the criteria
            for i in np.nonzero(points2)[0]:
                if points[i] >= pointslimit and points2[i] >= points2limit:
                    active[i] = 1
            points2limit = points2limit - 1
            if points2limit == -100:
                pointslimit = pointslimit - 1
                points2limit = pointslimit + 4
            if points2limit == 0:
                break
            if points2limit == pointslimit:
                pointslimit = -99 # then no limit at all
        if sum(active) == 0:
            break
        for i in np.nonzero(tiebreaker)[0]:
            if active[i] == 0:
                tiebreaker[i] = 0
    winners = []
    for i in np.where(tiebreaker==max(tiebreaker))[0]:
        winners.append(athletes[i])
    position = len(resultlist) + 1
    for winner in winners:
        resultlist.append([position,winner])
        athletes.remove(winner)
    break
print(resultlist)